In [ ]:
from tqdm import tqdm
# for문이 돌아가는 진행정도를 확인할 수 있다.
for i in tqdm(range(1,100)):
    print(i,end='')

## plotly 요즘 뜨는 그래프 모듈 - interaction한 시각화 표현 가능 -> export해서 링크에서 열 수 있다.
## bokeh

In [ ]:
import pandas as pd
import requests as re
from bs4 import BeautifulSoup as bs4
import csv
#request를 통해 html을 불러오면 bs4가 html을 짤라서 가져옴
#검색어 설정


result=pd.DataFrame(columns=['날짜','요일', '홈', '방문', '구장', '관중수'])
print(type(result))
url = "https://www.koreabaseball.com/History/Crowd/GraphDaily.aspx"
res = re.get(url)
res.raise_for_status()

#Send a GET request to the URL
if res.status_code ==200:
    soup = bs4(res.text, "lxml") # bs4에서 파서를 짜를때 html인지 xmln인지 웹 사이트에서 확인해야함.  xmln이면 lxml로 짜르면 됨.
    data_rows=soup.find('table',attrs={'class':'tData'})
    table=pd.read_html(str(data_rows))
    table = table[0]
    #table=table.dropna()
    result=pd.concat([result,table])
else:
    print('Failed to retrieve the web page') # 정책상 크롤링이 막혔을 경우 출력

result.drop(502,axis=0,inplace=True)

### 한글 폰트 설정

In [ ]:
from datetime import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
%matplotlib inline
# 한글 폰트 설정

plt.rc('font', family='NanumBarunGothic')

In [ ]:
from datetime import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt

#result['관중수']=result['관중수'].str.replace(',','') # 각 열의 ,을 공백으로 바꾸기
result['관중수']=result['관중수'].astype('int64') #문자열을 숫자열로 바꾸기
result['날짜']=pd.to_datetime(result['날짜'])
result['달별']=result['날짜'].dt.strftime('%m').astype('int64') # 날짜에서 월만 가지고 오기
result

In [ ]:
fig = plt.figure(figsize = (12, 4))
chart = fig.add_subplot(1,1,1)
new_result = result.set_index('날짜')
new_result=new_result['관중수']
chart.plot(new_result['2023-04':'2023-09'])


### 요일별 평균 관중수

In [ ]:
import numpy as  np
import matplotlib.pyplot as plt
import numpy as np
estimators = [(np.mean, 'Mean'), ## 평균
              (np.median, 'Median'), ## 중앙값
              (np.max, 'Max'), ## 최대값
              (np.min, 'Min') ## 최소값
             ]
sns.set_theme(style="whitegrid") ## figure, axes 스타일
fig, ax = plt.subplots(2, 2)
fig.set_size_inches((10,10))
fig.set_facecolor('white')
for i, e in enumerate(estimators):
    estimator = e[0]
    title = e[1]
    row = i//2
    col = i%2
    cur_ax = ax[row, col]
    sns.barplot(x="요일", y="관중수", data=result, estimator=estimator,
                ax=cur_ax, ## 현재 axes에 그려짐
                errwidth=0
               )
    cur_ax.set_title(title)

In [ ]:
import numpy as  np
import matplotlib.pyplot as plt
import numpy as np
estimators = [(np.mean, 'Mean'), ## 평균
              (np.max, 'Max'), ## 중앙값
              (np.std, 'Std'), ## 표준편차
              (np.min, 'Min') ## 최소값
             ]
sns.set_theme(style="whitegrid") ## figure, axes 스타일
fig, ax = plt.subplots(2, 2)
fig.set_size_inches((10,10))
fig.set_facecolor('white')
for i, e in enumerate(estimators):
    estimator = e[0]
    title = e[1]
    row = i//2
    col = i%2
    cur_ax = ax[row, col]
    sns.barplot(x="방문", y="관중수", data=result, estimator=estimator,
                ax=cur_ax, ## 현재 axes에 그려짐
                errwidth=0,
               )
    cur_ax.set_title(title)

In [ ]:
facet=sns.FacetGrid(result, col='달별',hue='방문') #tips데이터로 그래프 그리되, day를 기준으로 그래프 나누기
facet.map(sns.barplot,'관중수') # map(원하는 그래프,x축 rug: 밀집도 확인)


In [ ]:
result.groupby('요일')['관중수'].mean()

## Selenium


### 11번가 Best 500 아이템 정보 추출하기**

- 추출 항목 : 순번, 제품명, 가격, URL

- https://www.11st.co.kr/browsing/BestSeller.tmall?method=getBestSellerMain&xfrom=main^gnb

-> 컨트롤+쉬프트+c를 눌러도 개발자도구가 들어가진다.

In [ ]:
# 코랩을 시작할 때 아래코드를 한 번 돌려줍니다.
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

- Selenium webDriver(셀레니엄 웹드라이버) 는 많은 브라우저(FireFox,Edge,InternetExplorer,Chrome...) , 운영체제 및 프로그래밍언어(ruby,js,C#,JAVA,Python .. ) 등을 지원하며 웹 응용 프로그램들의 테스트 를 단순화 함과 동시에 가속화를 도와주는 툴입니다.


- colab에서는 chrome webdriver를 설치해줘야 합니다.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By # class 태그나, id태그의 속성값을 가져오려고 합니다.
import time

# colab에서 selenium을 돌리기 위한 옵션들
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # 우리 컴퓨터가 아니여서 필요한 옵션
chrome_options.add_argument('--no-sandbox') # 내부(제한된) 환경에서도 잘 작동하도록
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# 11번가 이동
driver.get('https://www.11st.co.kr/browsing/BestSeller.tmall?method=getBestSellerMain&xfrom=main%5Egnb#pageNum%%6')

In [ ]:
lists = driver.find_element(By.ID,'bestPrdList').find_elements(By.CLASS_NAME,'viewtype')

In [ ]:
#스크롤 다운
SCROLL_PAUSE_SEC = 1

#스크롤 높이 가져옴.
last_height = driver.execute_script('return document.body.scrolHeight')

while True:
    # 끝까지 스크롤 다운
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    #1초 대기
    time.sleep(SCROLL_PAUSE_SEC)

    #스크롤 다운 후 스크롤 높이 다시 가져오기
    new_height = driver.execute_script('return document.body.scrollHeight')
    if new_height==last_height:
        break
    last_height = new_height


In [ ]:
best=[]
product=[]
price=[]
url=[]


for list in lists:
    bestlist = list.find_elements(By.TAG_NAME, 'li')
    for item in bestlist:
        best.append(item.find_element(By.CLASS_NAME, 'best').text) #순번
        product.append(item.find_element(By.CLASS_NAME, 'pname').find_element(By.TAG_NAME, 'p').text) #제품명
        price.append(item.find_element(By.CLASS_NAME, 'sale_price').text) #가격
        url.append(item.find_element(By.CLASS_NAME, 'box_pd.ranking_pd').find_element(By.TAG_NAME, 'a').get_attribute('href')) #URL 추
        #중간에 스페이스 태그로 되어있는 것은 .으로 연결이 가능함.


df=pd.DataFrame({ 'best':best,
                 'Product_name':product,
                 'Price':price,
                 'url':url})

In [ ]:
df

### 셀레니움으로 쿠팡 웹 크롤링

In [ ]:
#headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
driver.get('https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page=1&rocketAll=false&searchIndexingToken=1=9&backgroundColor=')

In [ ]:
lists = driver.find_elements(By.TAG_NAME,'dl')
lists

In [ ]:
product=[]
price=[]
review_n=[]

for list1 in lists:
    bestlist=list1.find_elements(By.CLASS_NAME, 'descriptions')
    for item in bestlist:
        product.append(item.find_element(By.CLASS_NAME,'name').text) #제품명
        price.append(item.find_element(By.CLASS_NAME,'price-value').text) #순번
        #review_n.append(item.find_element(By.CLASS_NAME,'star').find_element(By.TAG_NAME,'em').text)


df=pd.DataFrame({'Product_name':product,
                 'Price':price,})

In [ ]:
df
print(review_n)

### Request로 쿠팡 웹 크롤링

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page=1&rocketAll=false&searchIndexingToken=1=6&backgroundColor="
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}
res = requests.get(url, headers=headers)
res.raise_for_status() # 웹페이지의 상태가 정상인지 확인

soup = BeautifulSoup(res.text, "lxml") # 가져온 HTML 문서를 파서를 통해 BeautifulSoup 객체로 만듦

items = soup.find_all("li", attrs={"class":re.compile("^search-product")}) # li 태그 중에서 class 옵션이 search-product로 시작하는 요소들만 가져온다.

In [ ]:
for item in items:

    name = item.find("div", attrs={"class":"name"}).get_text() # 제품명

    price = item.find("strong", attrs={"class":"price-value"}).get_text() # 가격

    rate = item.find("em", attrs={"class":"rating"}) # 평점
    if rate:
        rate = rate.get_text()
    else:
        rate = "평점 없음"

    rate_cnt = item.find("span", attrs={"class":"rating-total-count"}) # 평점 수
    if rate_cnt:
        rate_cnt = rate_cnt.get_text()
    else:
        rate_cnt = "평점 수 없음"

    print(name, price, rate, rate_cnt)

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36", "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"}

for i in range(1, 6):
    #print("페이지 :", i)
    url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&channel=user&component=&eventCategory=SRP&trcid=&traid=&sorter=scoreDesc&minPrice=&maxPrice=&priceRange=&filterType=&listSize=36&filter=&isPriceRange=false&brand=&offerCondition=&rating=0&page={}&rocketAll=false&searchIndexingToken=1=6&backgroundColor=".format(i)
    #1 ~ 5 페이지 반복하기

    res = requests.get(url, headers=headers)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, "lxml")

    items = soup.find_all("li", attrs={"class":re.compile("^search-product")})
    # print(items[0].find("div", attrs={"class":"name"}).get_text())
    for item in items:

        # 광고 제품은 제외
        ad_badge = item.find("span", attrs={"class":"ad-badge-text"})
        if ad_badge:
            #print("  <광고 상품 제외합니다>")
            continue

        name = item.find("div", attrs={"class":"name"}).get_text() # 제품명

        # 애플 제품 제외
        if "Apple" in name:
            #print("  <Apple 상품 제외합니다")
            continue

        price = item.find("strong", attrs={"class":"price-value"}).get_text() # 가격

        # 리뷰 100개 이상, 평점 4.5 이상 되는 것만 조회
        rate = item.find("em", attrs={"class":"rating"}) # 평점
        if rate:
            rate = rate.get_text()
        else:
            #rate = "평점 없음"
            #print("  <평점 없는 상품 제외합니다>")
            continue

        rate_cnt = item.find("span", attrs={"class":"rating-total-count"}) # 평점 수
        if rate_cnt:
            rate_cnt = rate_cnt.get_text()[1:-1] # 예 : (26), 괄호 없애기

        else:
            #rate_cnt = "평점 수 없음"
            #print("  <평점 수 없는 상품 제외합니다>")
            continue

        link = item.find("a", attrs={"class":"search-product-link"})["href"]

        if float(rate) >= 4.5 and int(rate_cnt) >= 100:
            #print(name, price, rate, rate_cnt)
            print(f"제품명 : {name}")
            print(f"가격 : {price}")
            print(f"평점 : {rate}점 ({rate_cnt})개")
            print("바로가기 : {}".format("https://www.coupang.com/"+link))
            print("-"*100)